<h2>Module 0</h2>
Import libraries

In [8]:
import caiman as cm
import numpy as np
import pickle
import shutil
import os
import numpy as np
from tifffile import imwrite
from glob import glob
from moviepy.editor import VideoFileClip
from caiman.source_extraction.cnmf import params
from caiman.motion_correction import MotionCorrect
from time import time
import warnings
warnings.filterwarnings('ignore')

root = 'F:\\Test\\'

<h2>Module 1</h2>
Manual video inspection

In [2]:
%run cd_cropper.py

Found 6 avi files: [WindowsPath('F:/Test/M22old/2023_03_21/13_28_12/Miniscope/0.avi'), WindowsPath('F:/Test/M22old/2023_03_21/13_28_12/Miniscope/1.avi'), WindowsPath('F:/Test/M22old/2023_03_21/13_28_12/Miniscope/2.avi'), WindowsPath('F:/Test/M22old/2023_03_21/13_28_12/Miniscope/3.avi'), WindowsPath('F:/Test/M22old/2023_03_21/13_28_12/Miniscope/4.avi'), WindowsPath('F:/Test/M22old/2023_03_21/13_28_12/Miniscope/5.avi')]
Data shape: (5926, 600, 600)
{'IMAGE_SHAPE': (5926, 600, 600), 'LEFT': 40, 'RIGHT': 30, 'UP': 40, 'DOWN': 30}
Saving croppings to F:\Test\M22old\2023_03_21\13_28_12\Miniscope\Miniscope_cropping.pickle


<h2>Module 2</h2>
Batch cropping and timestamp extraction

In [3]:
#Batch crop
pick_names = glob(root + '*\*\*\*\*cropping.pickle')

for name in pick_names:
    start = time()
    with open(name, 'rb') as f:
        cr_dict = pickle.load(f,)
    avi_names = glob(os.path.dirname(name) + '\\*.avi') 
    splt_path = os.path.normpath(name).split(os.sep)
    out_fname = root + '_'.join(splt_path[-5:-2]) + '_CR.tif'
    whole_data = []
    for av_name in avi_names:
        clip = VideoFileClip(av_name)
        data = np.array([frame[cr_dict['UP']:,cr_dict['LEFT']:,0] for frame in clip.iter_frames()])
        if cr_dict['DOWN']:
            data = data[:,:-cr_dict['DOWN'],:]
        if cr_dict['RIGHT']:
            data = data[:,:,:-cr_dict['RIGHT']]
        whole_data.append(data[:-1])
        #cropping per se
    imwrite(out_fname, np.concatenate(whole_data, axis=0), photometric='minisblack')
    print('_'.join(splt_path[-5:-2]) + f' cropped in {time() - start:.1f}s')
    
    #Extract and copy timestamp files
    tst_name = os.path.dirname(name) + '\\timeStamps.csv'
    out_fname = root + '_'.join(splt_path[-5:-2]) + '_timestamp.csv'
    shutil.copy(tst_name, out_fname)
    

M22old_2023_03_21_13_28_12 cropped in 27.3s
M25old_2023_03_21_13_55_02 cropped in 32.0s
M27old_2023_03_21_14_14_49 cropped in 29.4s


<h2>Module 3</h2>
Batch motion correction

In [4]:
#Batch motion correction
fnames = glob(root + '*_CR.tif')

mc_dict = {
    'pw_rigid': False,         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
    'max_shifts': (35, 35),    # maximum allowed rigid shift
    'gSig_filt': (8, 8),       # size of high pass spatial filtering, used in 1p data
    'strides': (48, 48),       # start a new patch for pw-rigid motion correction every x pixels
    'overlaps': (24, 24),      # overlap between pathes (size of patch strides+overlaps)
    'max_deviation_rigid': 15,  # maximum deviation allowed for patch with respect to rigid shifts
    'border_nan': 'copy',      # replicate values along the boundaries
    'use_cuda': True,          # Set to True in order to use GPU
    'memory_fact': 4,          # How much memory to allocate. 1 works for 16Gb, so 0.8 show be optimized for 12Gb.
    'niter_rig': 1,
    'splits_rig': 20,          # for parallelization split the movies in  num_splits chuncks across time
                               # if none all the splits are processed and the movie is saved
    'num_splits_to_process_rig': None} # intervals at which patches are laid out for motion correction  

for name in fnames:
    start = time()
    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    if 'dview' in locals():
        cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)
    
    opts = params.CNMFParams(params_dict=mc_dict)
    mc = MotionCorrect([name], dview=dview, **opts.get_group('motion'))
    mc.motion_correct(save_movie=True)
    fname_mc = mc.fname_tot_els if mc.pw_rigid else mc.fname_tot_rig
    if mc.pw_rigid:
        bord_px = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)), np.max(np.abs(mc.y_shifts_els)))).astype(np.uint8)
    else:
        bord_px = np.ceil(np.max(np.abs(mc.shifts_rig))).astype(np.uint8)
    mc.bord_px = 0 if mc.border_nan == 'copy' else bord_px
    mov = mc.apply_shifts_movie([name])
    
    imwrite(name[:-4] + '_MC.tif', np.array(mov, dtype='uint8'), photometric='minisblack')
    print(os.path.split(name)[-1] + f' motion corrected in {time() - start:.1f}s')
    try:      #some clean-up
        mmap_files = glob(root + '*.mmap')
        for mm in mmap_files:
            os.remove(mm)
    except:
        continue

100%|██████████| 1/1 [00:01<00:00,  1.78s/it]


M22old_2023_03_21_13_28_12_CR.tif motion corrected in 131.4s


100%|██████████| 1/1 [00:02<00:00,  2.32s/it]


M25old_2023_03_21_13_55_02_CR.tif motion corrected in 166.3s


100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


M27old_2023_03_21_14_14_49_CR.tif motion corrected in 184.5s


<h2>Module 4</h2>
Batch cnmf

In [9]:
fnames = glob(root + '*MC.tif')

# dataset dependent parameters
min_SNR = 3                    # adaptive way to set threshold on the transient size
r_values_min = 0.75            # threshold on space consistency 

param_dict={'fr': 20,                   # rame rate, frames per second
            'decay_time': 2,            # typical duration of calcium transient 
            'method_init': 'corr_pnr',  # use this for 1 photon
            'K': None,                  # upper bound on number of components per patch, in general None
            'gSig': (5, 5),             # gaussian HALF-width of a 2D gaussian kernel, which approximates a neuron
            'gSiz': (21, 21),           # average diameter of a neuron, in general 4*gSig+1
            'merge_thr': 0.85,          # merging threshold, max correlation allowed
            'p': 1,                     # order of the autoregressive system
            'tsub': 1,                  # downsampling factor in time for initialization
            'ssub': 1,                  # downsampling factor in space for initialization
            'rf': 30,                   # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
            'stride': 21,               # amount of overlap between the patches in pixels(keep it at least large as gSiz, i.e 4 times the neuron size gSig) 
            'only_init': True,          # set it to True to run CNMF-E
            'nb': 2,                    # number of background components (rank) if positive, else exact ring model with following settings
                                        # gnb= 0: Return background as b and W
                                        # gnb=-1: Return full rank background B
                                        # gnb<-1: Don't return background
            'nb_patch': 0,              # number of background components (rank) per patch if gnb>0, else it is set automatically
            'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
            'low_rank_background': None,           # None leaves background of each patch intact, True performs global low-rank approximation if gnb>0
            'update_background_components': True,  # sometimes setting to False improve the results
            'min_corr': .8,                        # min peak value from correlation image
            'min_pnr': 10,                         # min peak to noise ration from PNR image
            'normalize_init': False,               # just leave as is
            'center_psf': True,                    # leave as is for 1 photon
            'ssub_B': 2,                           # additional downsampling factor in space for background
            'ring_size_factor': 1.4,               # radius of ring is gSiz*ring_size_factor
            'del_duplicates': True,                # whether to remove duplicates from initialization
            'border_pix': 0}                       # number of pixels to not consider in the borders
opts = params.CNMFParams(params_dict = param_dict)

for name in fnames:
    start = time()
    if 'dview' in locals():
        cm.stop_server(dview=dview)
        dview.terminate()
    #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
    c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)
    cnm = cm.source_extraction.cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=None, params=opts)

    mem_fname = cm.save_memmap([name], base_name=str(int(time())), order='C', border_to_0=0, dview=dview)
    Yr, dims, T = cm.load_memmap(mem_fname)
    images = Yr.T.reshape((T,) + dims, order='F')
    cnm.fit(images)
    cnm.params.set('quality', {'min_SNR': min_SNR, 'rval_thr': r_values_min, 'use_cnn': False})
    cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
    
    _, pnr = cm.summary_images.correlation_pnr(images[::5], gSig=param_dict['gSig'][0], swap_dim=False)
    cnm.estimates.imax = (pnr*255/np.max(pnr)).astype('uint8')
    with open(name[:-4] + '_estimates.pickle', "wb") as f:
        pickle.dump(cnm.estimates, f)
              
    cm.stop_server(dview=dview)
    dview.terminate()
    print(os.path.split(name)[-1] + f' cnmf-ed in {time() - start:.1f}s')
    #Clean-up memmap files
    mm_files = glob(root + '*.mmap')
    for mm in mm_files:
        try:
            os.remove(mm)
        except:
            continue

M22old_2023_03_21_13_28_12_CR_MC.tif cnmf-ed in 235.0s
M25old_2023_03_21_13_55_02_CR_MC.tif cnmf-ed in 251.0s
M27old_2023_03_21_14_14_49_CR_MC.tif cnmf-ed in 267.6s


<h2>Module 5</h2>
User inspection of cnmf results

In [26]:
#Stuff needed for plotting and widget callbacks
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import LinearColorMapper, CDSView, ColumnDataSource, Plot, CustomJS, Button, IndexFilter
from bokeh.layouts import column, row
from bokeh.io import push_notebook

from cd_inspector_callbacks import *
output_notebook()

est_fnames = glob(root + '*estimates.pickle')
est = LoadEstimates(est_fnames[0])
p1, p2, src = DrawFigures(est)
i = 0

Loading BokehJS ...

In [27]:
#Plotting itself
button_load = Button(label="Load next data", button_type="success", width = 120)     
button_load.js_on_event('button_click', CustomJS(code="""
     var kernel = IPython.notebook.kernel;
     kernel.execute("i = i + 1");
     kernel.execute("est = LoadEstimates(est_fnames[i])");
     kernel.execute("p1, p2, src = DrawFigures(est)")
     kernel.execute("show(column(row(button_load, button_del, button_merge, button_seed, button_save), row(p1, p2)))")
     """)) 

button_del = Button(label="Delete selected", button_type="success", width = 120)     
button_del.js_on_event('button_click', CustomJS(args=dict(src = src), code="""
     var si = src.selected.indices;
     var kernel = IPython.notebook.kernel;
     kernel.execute("ind_to_del= " + si);
     kernel.execute("est = DeleteSelected(est, ind_to_del)");
     kernel.execute("src.data = EstimatesToSrc(est)");
     kernel.execute("push_notebook(handle = t)") """)) 

button_merge = Button(label="Merge selected", button_type="success", width = 120)     
button_merge.js_on_event('button_click', CustomJS(args=dict(src = src), code="""
     var si = src.selected.indices;
     var kernel = IPython.notebook.kernel;
     kernel.execute("ind_to_mrg = " + si);
     kernel.execute("est = MergeSelected(est, ind_to_mrg, opts)");
     kernel.execute("src.data = EstimatesToSrc(est)");
     kernel.execute("push_notebook(handle = t)") """))  

button_seed = Button(label="Seed new neurons", button_type="success", width = 120)     
button_seed.js_on_event('button_click', CustomJS(args=dict(src = src), code="""
     var kernel = IPython.notebook.kernel;
     kernel.execute("est = SeedContours(est)");
     kernel.execute("src.data = EstimatesToSrc(est)");
     kernel.execute("push_notebook(handle = t)") """))

button_save = Button(label="Save results", button_type="success", width = 120)
button_save.js_on_event('button_click', CustomJS(code="""
     var kernel = IPython.notebook.kernel;
     kernel.execute("SaveResults(est)") """))

t = show(column(row(button_load, button_del, button_merge, button_seed, button_save), row(p1, p2)), notebook_handle = 'True')

In [28]:
import tifffile as tfl
import caiman as cm
import pandas as pd
import numpy as np
import pickle
import os
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import LinearColorMapper, CDSView, ColumnDataSource, Plot, CustomJS, Button, IndexFilter
from bokeh.layouts import column, row
from bokeh.io import push_notebook
from glob import glob
from caiman.source_extraction.cnmf import params
from time import time
from scipy.ndimage import gaussian_filter
from scipy.io import savemat

def SaveResults(estimates, sigma = 3):
    with open(estimates.name.partition('.')[0] + '_final.pickle', "wb") as f:
        pickle.dump(estimates, f) #just in case, may be it's excessive
    #traces timestamping and writing
    stamped_traces = np.concatenate(([estimates.time], estimates.C[estimates.idx_components]), axis=0)
    pd.DataFrame(stamped_traces.T).to_csv(estimates.name.partition('estimates')[0] + 'traces_test.csv', index=False, header = ['time_s', *np.arange(len(estimates.idx_components))])

    #making directory and tiff writing
    fold = estimates.name.partition('estimates')[0] + 'filters_test'
    if not os.path.exists(fold):
        os.mkdir(fold)
    ims = []
    for i,sp in enumerate(estimates.A.T[estimates.idx_components]):
        im = sp.reshape(estimates.imax.shape[::-1]).todense()
        if sigma:    #gaussian smoothing of neural contours, omitted if sigma=0
            im = gaussian_filter(im, sigma=sigma)
        ims.append((im*255/np.max(im)).astype(np.uint8))
        tfl.imwrite(fold + f'\\filter_{i+1:03d}.tif', ims[-1])
    savemat(fold + '_session.mat', {"A":np.array(ims)})
    return

<h2>Module 6</h2>
Batch spike detection

In [ ]:
#TODO: trancscribe here sd_scripts.m
#option: deal with cnmf-detected spikes from module 4

#INPUT: (timestamped) cnmf raw traces (and mb spikes)
#OUTPUT: detected spikes as .csv files

<h2>Module 7</h2>
Export results to G-drive, clean-up in local storage

In [ ]:
#INPUT: url to G-drive storage, contours, raw traces, detected spikes

In [ ]:
#### Sandbox
pick_end = '_reselected.pickle'
                    
pick_fnames = glob(root + '*' + pick_end)
for name in pick_fnames:
    with open(name, "rb") as f:
        [rois, traces] = pickle.load(f,)
    tst = glob(name[:-len(pick_end)-6] + '*_timestamp.csv')[0]
    time_s = np.genfromtxt(tst, delimiter = ',', skip_header = 1)[:,1]/1000
    time_s = time_s[:traces.shape[1]]
    stamped_traces = np.concatenate(([time_s], traces), axis=0)
#    np.savetxt(name[:-len(pick_end)-6] + '_traces.csv', stamped_traces.T, delimiter=",")
    pd.DataFrame(stamped_traces.T).to_csv(name[:-len(pick_end)-6] + '_traces.csv', index=False, header = ['time_s', *np.arange(traces.shape[0])])
    print(f'Written {traces.shape[0]: 3} cells  {traces.shape[1]} frames in {name[:-len(pick_end)-6]}_traces.csv') 
    
    
    
pick_end = '_results.pickle'
                    
pick_fnames = glob(root + '*' + pick_end)
for name in pick_fnames:
    with open(name, "rb") as f:
        [rois, traces,a,b] = pickle.load(f,)
    tst = glob(name[:-len(pick_end)-6] + '*_timestamp.csv')[0]
    time_s = np.genfromtxt(tst, delimiter = ',', skip_header = 1)[:,1]/1000
    time_s = time_s[:traces.shape[1]]
    stamped_traces = np.concatenate(([time_s], traces), axis=0)
#    np.savetxt(name[:-len(pick_end)-6] + '_traces.csv', stamped_traces.T, delimiter=",")
    pd.DataFrame(stamped_traces.T).to_csv(name[:-len(pick_end)-6] + '_traces.csv', index=False, header = ['time_s', *np.arange(traces.shape[0])])
    print(f'Written {traces.shape[0]: 3} cells  {traces.shape[1]} frames in {name[:-len(pick_end)-6]}_traces.csv')      